In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from prophet import Prophet
import pickle

# Belirli tarih aralığı için input verilerinin tahmini ve Regresyon modeli ile satış tahmini

In [2]:
#Veri setini oku
df = pd.read_excel('/content/drive/MyDrive/Veri-Seti.xlsx')
df.head()

,Date,Otomotiv Satis,OTV Orani,Faiz,EUR/TL,Kredi Stok
0,2010-01-01,20095.0,37.0,13.1820,2.107385,341244.126
1,2010-02-01,31172.0,37.0,13.2725,2.073890,351940.949
2,2010-03-01,51769.0,37.0,12.7275,2.085522,361307.320
3,2010-04-01,54946.0,37.0,12.0120,2.009010,373575.340
4,2010-05-01,59377.0,37.0,11.7400,1.948870,387708.051


In [3]:
df['Date'] = pd.to_datetime(df['Date'])


In [4]:
train = df[df['Date'] < '2022-06-01']
test = df[df['Date'] >= '2022-06-01']
test

,Date,Otomotiv Satis,OTV Orani,Faiz,EUR/TL,Kredi Stok
149,2022-06-01,NaN,NaN,NaN,NaN,NaN
150,2022-07-01,NaN,NaN,NaN,NaN,NaN
151,2022-08-01,NaN,NaN,NaN,NaN,NaN
152,2022-09-01,NaN,NaN,NaN,NaN,NaN
153,2022-10-01,NaN,NaN,NaN,NaN,NaN
154,2022-11-01,NaN,NaN,NaN,NaN,NaN
155,2022-12-01,NaN,NaN,NaN,NaN,NaN
156,2023-01-01,NaN,NaN,NaN,NaN,NaN
157,2023-02-01,NaN,NaN,NaN,NaN,NaN
158,2023-03-01,NaN,NaN,NaN,NaN,NaN


# Input değerlerinin tahmini

In [ ]:
# Load the data and split into train and test

train = df[df['Date'] < '2022-06-01']
test = df[df['Date'] >= '2022-06-01']

# Create a Prophet model for each independent variable
otv_model = Prophet()
faiz_model = Prophet()
eur_model = Prophet()
kredi_model = Prophet()

# Rename the columns to fit Prophet's input format
otv_train = train[['Date', 'OTV Orani']].rename(columns={'Date': 'ds', 'OTV Orani': 'y'})
faiz_train = train[['Date', 'Faiz']].rename(columns={'Date': 'ds', 'Faiz': 'y'})
eur_train = train[['Date', 'EUR/TL']].rename(columns={'Date': 'ds', 'EUR/TL': 'y'})
kredi_train = train[['Date', 'Kredi Stok']].rename(columns={'Date': 'ds', 'Kredi Stok': 'y'})

# Fit each model on the training data
otv_model.fit(otv_train)
faiz_model.fit(faiz_train)
eur_model.fit(eur_train)
kredi_model.fit(kredi_train)

# Make predictions for the test data
otv_future = otv_model.make_future_dataframe(periods=len(test), freq='M')
otv_forecast = otv_model.predict(otv_future)
faiz_future = faiz_model.make_future_dataframe(periods=len(test), freq='M')
faiz_forecast = faiz_model.predict(faiz_future)
eur_future = eur_model.make_future_dataframe(periods=len(test), freq='M')
eur_forecast = eur_model.predict(eur_future)
kredi_future = kredi_model.make_future_dataframe(periods=len(test), freq='M')
kredi_forecast = kredi_model.predict(kredi_future)

# Extract the predicted values for the test data
otv_pred = otv_forecast.iloc[-len(test):]['yhat'].values
faiz_pred = faiz_forecast.iloc[-len(test):]['yhat'].values
eur_pred = eur_forecast.iloc[-len(test):]['yhat'].values
kredi_pred = kredi_forecast.iloc[-len(test):]['yhat'].values


# Satışların tahmini

In [ ]:
from sklearn.linear_model import LinearRegression

# Create a linear regression model
lr_model = LinearRegression()

# Prepare the data
X_train = train[['OTV Orani', 'Faiz', 'EUR/TL', 'Kredi Stok']]
y_train = train['Otomotiv Satis']

X_test = test[['OTV Orani', 'Faiz', 'EUR/TL', 'Kredi Stok']]
X_test['OTV Orani']=otv_pred
X_test['Faiz']=faiz_pred
X_test['EUR/TL']=eur_pred
X_test['Kredi Stok']=kredi_pred

# Fit the model on the training data
lr_model.fit(X_train, y_train)



In [6]:
X_test

,OTV Orani,Faiz,EUR/TL,Kredi Stok
149,55.675728,22.323957,13.604262,4.774598e+06
150,56.426420,22.570395,13.857084,4.849924e+06
151,57.208576,22.641987,14.438272,5.014128e+06
152,58.099822,23.199716,15.070368,5.083501e+06
153,57.636289,22.762387,15.227768,5.113763e+06
154,58.056922,22.462472,15.188928,5.168258e+06
155,56.348939,22.175890,15.382792,5.264543e+06
156,61.113556,23.772105,15.634341,5.293032e+06
157,61.036492,23.579509,15.732850,5.369647e+06
158,60.774784,22.988030,16.863977,5.579706e+06


In [7]:
y_pred = lr_model.predict(X_test)

# Print the predicted sales
print(y_pred)


[62003.6164496  62016.21789257 62109.09125426 59974.22110734
 60175.88944216 61997.77176097 61299.84446406 61697.55644664
 62509.69935346 61315.98655834 60871.95824362 61467.48585446
 62559.39169486]


In [ ]:
X_test['Date']=test['Date']
X_test['Otomotiv Satis']=y_pred

In [9]:
X_test.columns

Index(['OTV Orani', 'Faiz', 'EUR/TL', 'Kredi Stok', 'Date', 'Otomotiv Satis'], dtype='object')

In [10]:
X_test= X_test.reindex(['Date','OTV Orani', 'Faiz', 'EUR/TL', 'Kredi Stok','Otomotiv Satis'], axis=1)

In [11]:
X_test

,Date,OTV Orani,Faiz,EUR/TL,Kredi Stok,Otomotiv Satis
149,2022-06-01,55.675728,22.323957,13.604262,4.774598e+06,62003.616450
150,2022-07-01,56.426420,22.570395,13.857084,4.849924e+06,62016.217893
151,2022-08-01,57.208576,22.641987,14.438272,5.014128e+06,62109.091254
152,2022-09-01,58.099822,23.199716,15.070368,5.083501e+06,59974.221107
153,2022-10-01,57.636289,22.762387,15.227768,5.113763e+06,60175.889442
154,2022-11-01,58.056922,22.462472,15.188928,5.168258e+06,61997.771761
155,2022-12-01,56.348939,22.175890,15.382792,5.264543e+06,61299.844464
156,2023-01-01,61.113556,23.772105,15.634341,5.293032e+06,61697.556447
157,2023-02-01,61.036492,23.579509,15.732850,5.369647e+06,62509.699353
158,2023-03-01,60.774784,22.988030,16.863977,5.579706e+06,61315.986558


# Modellerin Kaydedilmesi

In [13]:
with open('otv_model.pkl', 'wb') as f:
    pickle.dump(otv_model, f)
with open('faiz_model.pkl', 'wb') as f:
    pickle.dump(faiz_model, f)
with open('eur_model.pkl', 'wb') as f:
    pickle.dump(eur_model, f)
with open('kredi_model.pkl', 'wb') as f:
    pickle.dump(kredi_model, f)
with open('lr_model.pkl', 'wb') as f:
    pickle.dump(lr_model, f)

